# Introducción a MicroPython

---

*Esta Notebook para funcionar necesita el Kernel [Jupyter MicroPython Kernel](https://pypi.org/project/jupyter_micropython_kernel/), instalable vía `pip install jupyter_micropython_kernel`. Los comandos soportados son:*

- *`%serialconnect` para conectarse a la placa*
- *`%esptool` para flashear el dispositivo*
- *`%lsmagic` para listar los comandos*

*La Notebook fue confeccionada mediante material original del Laboratorio de Instrumentación Virtual y Robótica Aplicada (LIVRA) de la Facultad de Ingeniería de la Universidad Nacional de Mar del Plata.*

---

## 0. Introducción

[MicroPython](https://micropython.org/) es una implementación del lenguaje de programación Python 3, escrita en C, optimizada para poder ejecutarse en un microcontrolador. MicroPython lo creó originalmente el programador y físico australiano Damien George, después de una exitosa campaña de Kickstarter que apoyó el proyecto en 2013. Aunque durante la campaña original de Kickstarter se lanzó MicroPython en conjunción con la placa de microcontrolador PyBoard, en la actualidad MicroPython soporta un amplio número de arquitecturas basadas en ARM.

MicroPython es un compilador completo del lenguaje Python a bytecode y un motor e intérprete en tiempo de ejecución del bytecode, que funciona en el hardware del microcontrolador. Al usuario se le presenta una línea de órdenes interactiva (el REPL) que soporta la ejecución inmediata de órdenes. Se incluye una selección de bibliotecas fundamentales de Python: MicroPython incluye módulos que permiten al programador el acceso al hardware en bajo nivel.

La documentación oficial se encuentra en [https://docs.micropython.org/en/latest/index.html](https://docs.micropython.org/en/latest/index.html).

## 1. La placa ESP32

Trabajaremos con la placa ESP32, la referencia rápida oficial se encuentra en [https://docs.micropython.org/en/latest/esp32/quickref.html](https://docs.micropython.org/en/latest/esp32/quickref.html), y el pinout es el siguiente:

![](ESP32%20Pinout.jpg)

No todos los GPIO son accesibles en todas las placas de desarrollo, pero cada GPIO específico funciona de la misma manera independientemente de la placa de desarrollo que estés usando. Si recién estás comenzando con el ESP32, te recomendamos leer la guía: [ESP32 Pinout Reference: Which GPIO pins should you use?](https://randomnerdtutorials.com/esp32-pinout-reference-gpios/)

| GPIO | Input      | Output | Notes                                                                      |
|------|------------|--------|----------------------------------------------------------------------------|
| 0    | pulled up  | OK     | outputs PWM signal at boot, must be LOW to enter flashing mode             |
| 1    | TX pin     | OK     | debug output at boot                                                       |
| 2    | OK         | OK     | connected to on-board LED, must be left floating or LOW to enter flashing mode |
| 3    | OK         | RX pin | HIGH at boot                                                               |
| 4    | OK         | OK     |                                                                            |
| 5    | OK         | OK     | outputs PWM signal at boot, strapping pin                                  |
| 6    | x          | x      | connected to the integrated SPI flash                                      |
| 7    | x          | x      | connected to the integrated SPI flash                                      |
| 8    | x          | x      | connected to the integrated SPI flash                                      |
| 9    | x          | x      | connected to the integrated SPI flash                                      |
| 10   | x          | x      | connected to the integrated SPI flash                                      |
| 11   | x          | x      | connected to the integrated SPI flash                                      |
| 12   | OK         | OK     | boot fails if pulled high, strapping pin                                   |
| 13   | OK         | OK     |                                                                            |
| 14   | OK         | OK     | outputs PWM signal at boot                                                 |
| 15   | OK         | OK     | outputs PWM signal at boot, strapping pin                                  |
| 16   | OK         | OK     |                                                                            |
| 17   | OK         | OK     |                                                                            |
| 18   | OK         | OK     |                                                                            |
| 19   | OK         | OK     |                                                                            |
| 21   | OK         | OK     |                                                                            |
| 22   | OK         | OK     |                                                                            |
| 23   | OK         | OK     |                                                                            |
| 25   | OK         | OK     |                                                                            |
| 26   | OK         | OK     |                                                                            |
| 27   | OK         | OK     |                                                                            |
| 32   | OK         | OK     |                                                                            |
| 33   | OK         | OK     |                                                                            |
| 34   | OK         |        | input only                                                                 |
| 35   | OK         |        | input only                                                                 |
| 36   | OK         |        | input only                                                                 |
| 39   | OK         |        | input only                                                                 |

Para conectar la placa a esta Notebook, hay que correr el siguiente comando:

In [ ]:
# Conectarse a la placa
%serialconnect

## 2. Entradas/Salidas

Para acceder a los pines de entrada/salida de la ESP32, hay que importar del módulo `machine` la definición correspondiente. Más información en [machine — functions related to the hardware](https://docs.micropython.org/en/latest/library/machine.html).

In [ ]:
from machine import Pin

# Crear pines de entrada
pin_5 = Pin(5, Pin.IN)              # Pin en modo entrada
pin_4 = Pin(4, Pin.IN, Pin.PULL_UP) # Habilita el resistor pull-up interno

# Crear pines de salida
pin_2 = Pin(2, Pin.OUT)
pin_0 = Pin(0, Pin.OUT, value=1)    # Coloca el pin en alto al crearlo

pin_2.value(0)

Al crear el objeto `pin`, se deben pasar como parámetros el número de pin y su modo (entrada o salida). En caso de que se ingrese solo el número de pin, se asume que es una entrada (que también puede explicitarse con `Pin.IN`).

Para acceder o modificar su valor, se puede usar el método `value()` de la siguiente manera:

In [ ]:
pin_status = pin_0.value() # Devuelve el valor del pin
print("El pin se encuentra en el estado:", pin_status)


In [ ]:
# Coloca en alto el pin 2
pin_2.value(1)

In [ ]:
# Coloca en bajo el pin 2
pin_2.off()

In [ ]:
# Coloca en alto el pin 2
pin_2.on()

También podemos configurar una salida como PWM (modulación de ancho de pulso). Para esto usamos la función PWM definida en el módulo `machine` y pasamos como argumento uno de los pines compatibles.

In [ ]:
from machine import PWM, Pin

pwm = PWM(Pin(2))
pwm.freq(10000) # Configura la frecuencia a 10kHz, acepta valores desde 1Hz a 40MHz
pwm.duty(512)   # Configura el duty cycle al 50% (0 - 1023)

# También podemos configurarlo todo en una sola línea
# pwm = PWM(Pin(2, freq=10000, duty=512))

# Podemos ver la configuración actual imprimiendo el objeto pwm
print(pwm)

In [ ]:
# Obtenemos el duty cycle actual
duty = pwm.duty()
print("El duty cycle es:", duty)

In [ ]:
# Obtenemos el duty cycle en nanosegundos
duty_ns = pwm.duty_ns()
print("El duty cycle en nanosegundos es:", duty_ns)

In [ ]:
# Detenemos la salida
pwm.deinit()

## 3. Control de tiempo

Podemos usar el módulo `time` que nos permite contar tiempos para poder realizar pausas o medir tiempo entre operaciones:

In [ ]:
import time

time.sleep(1)               # espera 1 segundo
time.sleep_ms(1000)         # ídem a anterior pero en milisegundos
time.sleep_us(1000000)      # ídem a anterior pero en microsegundos

inicio = time.ticks_ms()    # almacena la cantidad de ms
                            # desde el inicio del dispositivo

# se ejecutan varias operaciones que toman tiempo
time.sleep_ms(1000)

fin = time.ticks_ms()

# Se muestra la diferencia de tiempos en la misma unidad (ms)
print(time.ticks_diff(fin, inicio))

El código "Blink" de MicroPython es:

In [ ]:
from machine import Pin
import time

led_builtin = Pin(2, Pin.OUT)

while True:
    print("Ejecutando...");
    led_builtin.value(1)
    time.sleep(1)
    led_builtin.value(0)
    time.sleep(1)

También se pueden programar tareas repetitivas o después de pasado determinado tiempo usando los Timers internos de la ESP32. Se pueden acceder a cada Timer mediante su ID (del 0 a 3 inclusive): `tim = Timer(ID)`

In [ ]:
from machine import Timer

def tarea_repetitiva(timer):
    print("Tarea invocada por el timer", timer)

tim = Timer(0)
tim.init(period=1000, mode=Timer.PERIODIC,
        callback=tarea_repetitiva)

In [ ]:
# Detener el timer
tim.deinit()

En este ejemplo, se inicializa un timer que ejecuta una función de forma periódica cada 1 segundo (1000 ms). La función debe incorporar un parámetro que corresponde al timer que la ejecutó. Alternativamente, se puede cambiar el modo `mode=Timer.ONE_SHOT` que hace que se ejecute una sola vez pasado el tiempo indicado por `period`.

In [ ]:
def tarea_oneshot(timer):
    print("Tarea ONE_SHOT invocada por el timer", timer)

tim = Timer(0)
tim.init(period=1000, mode=Timer.ONE_SHOT, callback=tarea_oneshot)

## 4. Sensores, actuadores y periféricos

### 4.1. Control de servomotor

Utilizando el concepto de PWM visto anteriormente, es posible controlar el ángulo de giro de un servomotor según el ciclo de trabajo. Para esto, primero se configura una salida PWM a `50Hz` y los anchos de pulso según el ángulo requerido. A modo de referencia, deberían tomar aproximadamente los siguientes valores:

- Para la posición 0°: pulso de 1ms
- Para la posición 90°: pulso de 1.5ms
- Para la posición 180°: pulso de 2ms

Luego se ajusta el ciclo de trabajo acorde a la resolución del PWM (0 a 1023) y, teniendo en cuenta que 0ms se configura con 0 y 20ms es el pulso completo, se obtienen los siguientes valores:

- Para la posición 0°: ciclo de trabajo 51
- Para la posición 90°: ciclo de trabajo 76
- Para la posición 180°: ciclo de trabajo 102

In [ ]:
from machine import PWM, Pin
from time import sleep_ms

pwm = PWM(Pin(14))
pwm.freq(50)

for i in range(51, 103):
    pwm.duty(i)
    sleep_ms(10)

### 4.2. Conversión analógico-digital

El ESP32 contiene puertos de conversión de entrada analógica a digital ubicados en los pines `32` a `39` (bloque 1) y los pines `0`, `2`, `4`, `12` a `15` y `25` a `27` (bloque 2). Sin embargo, el bloque 2 es usado también por WiFi por lo que no se pueden usar en simultáneo.

Para configurar un pin, se lo pasamos a la función `ADC` del módulo `machine` de la misma manera que con PWM. Luego podemos leer su valor con el método `read()` o `read_uv()` para obtener la tensión en μV. También es posible ajustar el factor de atenuación para ampliar el rango de conversión:

- `ADC.ATTN_0DB`: 100mV - 950mV
- `ADC.ATTN_2_5DB`: 100mV - 1250mV
- `ADC.ATTN_6DB`: 150mV - 1750mV
- `ADC.ATTN_11DB`: 150mV - 2450mV

In [ ]:
from machine import ADC

# adc = ADC(Pin(35), atten=ADC.ATTN_11DB)
adc = ADC(Pin(35))

# Devuelve el valor ADC sin procesar según la resolución del bloque
# Por ejemplo, 0-4095 para una resolución de 12 bits.
lectura = adc.read()

# Devuelve el valor analógico sin procesar en el rango 0-65535
# lectura = adc.read_u16() 
# Devuelve el valor analógico en microvoltios
# lectura = adc.read_uv()
print(lectura)

### 4.3. DAC

Tambien es posible generar valores analógicos arbitrarios utilizando el conversor digital-analógico integrado al ESP32. Este nos permite convertir valores digitales de 8 bits a tensión de salida. La funcionalidad DAC está disponible en los pines `25` y `26`. En el ESP32S2 está disponible en los pines `17` y `18`.

In [ ]:
from machine import Pin, DAC

dac = DAC(Pin(25))
dac.write(127)

### 4.4. Sensor de temperatura y humedad DHT11/22

El lenguaje MicroPython incorpora una implementación del driver de los dispositivos DHT11 y DHT22.

In [ ]:
from dht import DHT11
from machine import Pin

dht = DHT11(Pin(32))
dht.measure()

print(f"Temperatura: {dht.temperature()} °C")
print(f"Humedad: {dht.humidity()} %")

### 4.5. NeoPixel

También se incluye el driver para LEDs WS2812B, también conocidos como NeoPixel.

In [ ]:
from neopixel import NeoPixel
from machine import Pin

# Create un NeoPixel: Pin, # pixels
np = NeoPixel(Pin(27), 1)

# Orden: G, R, B
np[0] = (0, 255, 0)

# Se aplican los cambios
np.write()

### 4.6. Pantalla LCD

Ejemplo para controlar una pantalla LCD en modo **paralelo**. El pinout es el siguiente:

| LCD Pin | ESP32 Pin (GPIO) | Descripción      |
|---------|-------------------|------------------|
| VSS     | GND               | Tierra           |
| VDD     | 3.3V o 5V         | Alimentación     |
| VO      | Potenciómetro o GND | Contraste       |
| RS      | GPIO 25           | Registro select  |
| RW      | GND               | Escritura        |
| E       | GPIO 26           | Enable           |
| D4      | GPIO 16           | Datos (bit 4)    |
| D5      | GPIO 17           | Datos (bit 5)    |
| D6      | GPIO 18           | Datos (bit 6)    |
| D7      | GPIO 19           | Datos (bit 7)    |

Crear un archivo `lcd.py` para realizar la implementación:

In [ ]:
from machine import Pin
from time import sleep

class LCD:
    def __init__(self, rs, enable, d4, d5, d6, d7):
        self.rs = Pin(rs, Pin.OUT)
        self.enable = Pin(enable, Pin.OUT)
        self.data_pins = [
            Pin(d4, Pin.OUT),
            Pin(d5, Pin.OUT),
            Pin(d6, Pin.OUT),
            Pin(d7, Pin.OUT)
        ]
        self.init_lcd()

    def pulse_enable(self):
        self.enable.value(1)
        sleep(0.0001)
        self.enable.value(0)
        sleep(0.0001)

    def send_nibble(self, nibble):
        for i in range(4):
            self.data_pins[i].value((nibble >> i) & 0x01)
        self.pulse_enable()

    def send_byte(self, byte, mode):
        self.rs.value(mode)
        self.send_nibble(byte >> 4)  # High nibble
        self.send_nibble(byte & 0x0F)  # Low nibble

    def command(self, cmd):
        self.send_byte(cmd, 0)

    def write_char(self, char):
        self.send_byte(ord(char), 1)

    def init_lcd(self):
        sleep(0.02)
        self.send_nibble(0x03)
        sleep(0.005)
        self.send_nibble(0x03)
        sleep(0.0001)
        self.send_nibble(0x03)
        self.send_nibble(0x02)

        self.command(0x28)  # 4-bit mode, 2 lines, 5x8 dots
        self.command(0x0C)  # Display on, cursor off
        self.command(0x01)  # Clear display
        sleep(0.002)

    def clear(self):
        self.command(0x01)
        sleep(0.002)

    def putstr(self, string):
        for char in string:
            self.write_char(char)

El funcionamiento es el siguiente:

1. `init_lcd` configura el LCD en modo de 4 bits.
2. `putstr` envía caracteres al LCD.
3. Los comandos como `0x01` (borrar pantalla) y `0x0C` (activar pantalla) controlan el comportamiento del LCD.

Ejemplo para testear la pantalla:

In [ ]:
from lcd import LCD
from time import sleep

# Configura los pines del ESP32
lcd = LCD(rs=25, enable=26, d4=16, d5=17, d6=18, d7=19)

# Mensaje en la pantalla LCD
lcd.putstr("Hola, Mundo!")
sleep(2)

# Borrar y mostrar un nuevo mensaje
lcd.clear()
lcd.putstr("MicroPython!!!")

## 5. Almacenamiento

### 5.1. Almacenamiento flash

La ESP32 no tiene una EEPROM interna física como algunas placas Arduino, pero es posible simular una pequeña memoria EEPROM usando el almacenamiento flash para almacenar datos persistentes. Mediante el módulo `os` es posible escribir y leer archivos en la memoria flash de la ESP32:

In [ ]:
# Escribir datos en un archivo
with open("data.txt", "w") as f:
    f.write("12345")  # Datos a almacenar
    
# Leer datos del archivo
with open("data.txt", "r") as f:
    datos = f.read()
    print("Datos leídos:", datos)

### 5.2. EEPROM externa

Para trabajar con una EEPROM externa, como la 24LC256, se utiliza el bus I2C para leer y escribir datos.

In [ ]:
from machine import I2C, Pin

# Configurar I2C
i2c = I2C(1, scl=Pin(22), sda=Pin(21), freq=400000)
direccion_eeprom = 0x50  # Dirección I2C de la EEPROM

# Escribir un byte (0x42) en la dirección 0x0000
i2c.writeto_mem(direccion_eeprom, 0x0000, b'\x42')

# Leer un byte desde la dirección 0x0000
data = i2c.readfrom_mem(direccion_eeprom, 0x0000, 1)
print("Dato leído de EEPROM:", data)

## 6. Comunicaciones Serial

Ejemplos de uso de *UART*, *I2C* y *SPI* para comunicarse con otros dispositivos.

### 6.1. UART

La comunicación UART es esencial para enviar y recibir datos entre la ESP32 y otros dispositivos seriales. En MicroPython, la comunicación UART se maneja mediante el módulo `machine` y su clase `UART`.

In [ ]:
from machine import UART, Pin

# Configura el UART 1 con un baudrate de 9600, pines TX en 17 y RX en 16
uart1 = UART(1, baudrate=9600, tx=Pin(17), rx=Pin(16))

# Enviar datos
uart1.write('Hola desde ESP32!')

# Leer datos recibidos (si hay datos disponibles)
if uart1.any():
    datos = uart1.read()  # Lee todos los datos disponibles
    print('Datos recibidos:', datos)

Para probar la comunicación bidireccional (echo), puedes implementar un eco serial. Este programa lee datos y los reenvía al dispositivo de origen:

In [ ]:
while True:
    if uart1.any():
        recibido = uart1.read()
        uart1.write(recibido)  # Enviar de vuelta los datos recibidos
        print('Echo:', recibido)

### 6.2. I2C

La comunicación I2C es útil para conectar sensores y otros periféricos. En MicroPython, se configura mediante el módulo `machine` y la clase `I2C`.

In [ ]:
from machine import I2C, Pin

# Configura el I2C en modo maestro con pines de SDA (21) y SCL (22)
i2c = I2C(1, scl=Pin(22), sda=Pin(21), freq=400000)

# Escanea dispositivos en el bus I2C
dispositivos = i2c.scan()
print('Dispositivos I2C encontrados:', dispositivos)

A continuación, se muestra un ejemplo genérico para leer datos de un sensor I2C en la dirección `0x40`:

In [ ]:
# Dirección del sensor
direccion_sensor = 0x40

# Leer 2 bytes de datos desde el registro 0xE3 del sensor
data = i2c.readfrom_mem(direccion_sensor, 0xE3, 2)

# Convertir los datos en temperatura
# Fórmula genérica, ajustar según el sensor
temp = (data[0] << 8 | data[1]) * 0.00268127
print('Temperatura:', temp, '°C')

### 6.3. SPI

La interfaz SPI (Serial Peripheral Interface) permite la comunicación rápida entre la ESP32 y otros dispositivos como pantallas y memorias flash. En MicroPython, el módulo `machine` contiene la clase `SPI` para configurar esta comunicación.

In [ ]:
from machine import Pin, SPI

# Configura SPI con los pines de SCK, MOSI, MISO y CS
spi = SPI(1, baudrate=1000000, polarity=0, phase=0, sck=Pin(18), mosi=Pin(23), miso=Pin(19))

# Pin de Chip Select (CS)
cs = Pin(5, Pin.OUT)

# Comunicación con un dispositivo SPI
cs.value(0)  # Habilitar el dispositivo
spi.write(b'\x9F')  # Comando de ejemplo para leer el ID del dispositivo
id_dispositivo = spi.read(3)  # Leer 3 bytes
cs.value(1)  # Deshabilitar el dispositivo

print('ID del dispositivo:', id_dispositivo)

Lectura y escritura en un dispositivo SPI (ej. memoria flash):

In [ ]:
# Comando de escritura de datos
cs.value(0)
spi.write(b'\x02')  # Comando de escritura
spi.write(b'\x00\x00\x00')  # Dirección de memoria
spi.write(b'Hello')  # Datos a escribir
cs.value(1)

# Comando de lectura de datos
cs.value(0)
spi.write(b'\x03')  # Comando de lectura
spi.write(b'\x00\x00\x00')  # Dirección de memoria
data = spi.read(5)  # Leer 5 bytes
cs.value(1)

print('Datos leídos:', data)

## 7. Bluetooth Low Energy (BLE)

Bluetooth Low Energy, BLE (también llamado Bluetooth Smart), es una variante de Bluetooth con ahorro de energía. La aplicación principal de BLE es la transmisión a corta distancia de pequeñas cantidades de datos (ancho de banda bajo). A diferencia de Bluetooth, que siempre está activado, BLE permanece en modo de suspensión constantemente, excepto cuando se inicia una conexión, esto hace que consuma muy poca energía.

Para funcionar, se necesitan los siguientes módulos:

- `ble_advertising.py`
- `ble_simple_peripheral.py`

In [ ]:
# This example demonstrates a UART periperhal.

import bluetooth
import random
import struct
import time

from ble_simple_peripheral import BLESimplePeripheral
from ble_advertising import advertising_payload
from micropython import const

ble = bluetooth.BLE()
p = BLESimplePeripheral(ble)

# Recepción de datos
def on_rx(v):
    print("RX", v)

p.on_write(on_rx)

# Envio de datos
i = 0
while True:
    if p.is_connected():
        # Short burst of queued notifications.
        for _ in range(3):
            data = str(i) + "_"
            print("TX", data)
            p.send(data)
            i += 1
    time.sleep_ms(1000)

**Funcionamiento:**

La función `on_rx(v)` es un callback que se ejecuta cuando el periférico BLE recibe datos del cliente conectado, imprime los datos recibidos `(v)`. Con `p.on_write(on_rx)` se asocia la función de recepción de datos al periférico BLE.

Luego se utiliza un bucle infinito para mantener la funcionalidad del periférico:

- `p.is_connected()` verifica si un cliente BLE está conectado al periférico.
- Se envían tres notificaciones consecutivas al cliente mediante `p.send(data)`.

## 8. WiFi

Para conectarnos a una red o crear una red propia en la ESP32, podemos usar el módulo `network`.

### 8.1. ESP32 como Access Point

In [ ]:
import network

ap = network.WLAN(network.AP_IF)    # se configura el modo AP
ap.config(essid="nombre de red")
ap.config(max_clients=4)            # cantidad máxima de conexiones
ap.active(True)

### 8.2. Conexión a una red existente

Para conectarnos a una red existente, podemos hacerlo interactivamente desde la línea de comandos para comprender cómo es el proceso:

In [ ]:
import network

In [ ]:
wlan = network.WLAN(network.STA_IF)

In [ ]:
wlan.active(True)

En primer lugar, se crea el objeto `wlan` con el cual usaremos para conectarnos a una red WiFi. Con el método `active()` activamos o desactivamos la red. Una vez activa, podemos escanear las redes visibles:

In [ ]:
wlan.scan()

Después de un tiempo, se genera una lista de redes con sus nombres, potencias y otros parámetros. Para ver una simple lista con los nombres, podemos usar un ciclo:

In [ ]:
for red in wlan.scan():
    print(red[0])

Para conectarnos con la red, se puede usar el método `connect`:

In [ ]:
wlan.connect("ssid", "password")

Luego de un tiempo, podemos verificar si se conectó exitosamente:

In [ ]:
wlan.isconnected()
# Conexion exitosa: True

In [ ]:
wlan.ifconfig()[0]
# Dirección IP asignada

Para automatizar este proceso y pueda ser ejecutado en la ESP32 y bloquee la ejecución hasta que logre conectarse, se puede usar el siguiente script:

In [ ]:
import network
from time import sleep_ms

wlan = network.WLAN(network.STA_IF)
if not wlan.active():
    wlan.active(True)

if not wlan.isconnected():
    wlan.connect("ssid", "password")

    print("Conectando...")
    while not wlan.isconnected():
        sleep_ms(1000)

config = wlan.ifconfig()
print(f"Conectado con ip {config[0]}")

## 9. MQTT

MQTT es un protocolo de mensajería estandarizado que distribuye la información a través de publicación y suscripción (publish/subscribe) a determinado tema (topic) y es eficiente para situaciones que se transporta poca información ya que consume poco ancho de banda.

Cada dispositivo se puede comportar como publicador o suscriptor en forma simultánea. La comunicación es manejada por un servidor, también denominado broker. Para dirigir la comunicación, se utilizan topics, los cuales, para este curso concreto, pueden seguir la siguiente convención:

- Único dispositivo: `/<parámetro>`
- Múltiples dispositivos: `/<dispositivo>/<parámetro>`

A modo de ejemplo, estos topics pueden ser:

- `/humedad`
- `/cocina/temperatura`

El cliente MQTT no está disponible de forma nativa para MicroPython, por lo que es necesario instalarlo. Para esto, se utiliza el gestor de paquetes `upip` desde la línea de comandos de un dispositivo ESP32 con MicroPython y, a la vez, conectado a una red WiFi con acceso a internet.

In [ ]:
import upip

In [ ]:
upip.install("micropython-umqtt.simple")

In [ ]:
upip.install("micropython-umqtt.robust")

El cliente *simple* implementa el protocolo MQTT, mientras que el cliente *robust* construye sobre el cliente simple un mecanismo de reconexión en caso de pérdida de conexión al servidor. Para crear el cliente se puede ejecutar el siguiente script:

In [ ]:
from umqtt.robust import MQTTClient

cliente = MQTTClient("nombre", "servidor", keepalive=30)
print("Conectando con servidor MQTT...")

cliente.connect(clean_session=False)
print("Conectado")

De esta manera, se establece una conexión al servidor indicando el nombre del dispositivo, y con `keepalive` se especifica en segundos un *heartbeat* para mantener la conexión activa. Es importante tener en cuenta que los nombres de los dispositivos son únicos en la red por lo que es recomendable utilizar nombres distintos para cada conexión. Luego, se establece una conexión con `clean_session=False` para garantizar persistencia en el caso que se desconecte y sea necesario reconectar, como las suscripciones activas y mensajes pendientes (si el cliente usa QoS 1 o QoS 2).

Es posible utilizar la MAC del dispositivo como `nombre`:

In [ ]:
# Obtener la dirección MAC
mac = wlan.config('mac')
mac_address = ':'.join('{:02x}'.format(b) for b in mac)
print("Dirección MAC:", mac_address)

### 9.1. Suscripciones

Una vez conectado el cliente, se puede suscribir a diferentes topic de forma tal que estará constantemente esperando la llegada de nuevos mensajes.

Una vez que llegue un mensaje nuevo, se ejecuta una función callback que permita decidir qué hacer según el mensaje.

Para suscribirse a un topic se usa el método subscribe y como parámetro un topic. Para conectarse a más topics, se puede repetir la invocación. Por otro lado, para escuchar todo los topics se puede utilizar `#` o `/jerarquía/#` para escuchar topics que pertenezcan a determinada jerarquía.

También hay que definir la función `callback`, la cual es una función de Python con dos parámetros: `topic` y `msg` los cuales llegan en formato bytes y pueden ser convertidos a cadena de texto con el método `decode()`.

Finalmente, se crea un bucle infinito dentro del cual se realiza la revisión de mensajes nuevos con el método del cliente `check_msg()`.

A continuación se muestra un script de cómo incorporar esto al código anterior.

In [ ]:
from umqtt.robust import MQTTClient
from time import sleep_ms

def callback(topic, msg):
    topic = topic.decode()
    msg = msg.decode()

    if topic == "/servidor":
        print(f"Llegó {msg} de {topic}")

cliente = MQTTClient("nombre", "servidor", keepalive=30)
print("Conectando a servidor MQTT...")
cliente.set_callback(callback)
cliente.connect(clean_session=False)
print("Conectado")
cliente.subscribe("#")

while True:
    cliente.check_msg()
    sleep_ms(500)

### 9.2. Publicaciones

Para publicar dentro de un topic, no es necesaria ninguna configuración extra. Solo se utiliza el método del cliente `publish` con argumentos `topic` y `mensaje`.

In [ ]:
cliente.publish("topic", "mensaje")

## 10. Ejemplo de aplicación: Adafruit IO

[Adafruit IO](https://io.adafruit.com/) es una plataforma para acercar a todos el mundo del Internet de las cosas (IoT). Es una forma más sencilla de transmitir, registrar e interactuar con sus datos.

In [ ]:
import network, urandom
from umqtt.robust import MQTTClient
from machine import Timer, Pin
from time import sleep_ms

#--- Credenciales ------------------------------------#
wifi_ssid = "ssid"
wifi_password = "password"

mqtt_server = "io.adafruit.com"
mqtt_user = "user"
mqtt_password = "aio_key"
mqtt_topic_1 = "{user}/feeds/{feedname}"
mqtt_topic_2 = "{user}/feeds/{feedname}"

#--- Setup ------------------------------------------#
pin_2 = Pin(2, Pin.OUT, value=0)

#--- Conexión a WiFi --------------------------------#
wlan = network.WLAN(network.STA_IF)
if not wlan.active():
    wlan.active(True)

if not wlan.isconnected():
    wlan.connect(wifi_ssid, wifi_password)

    print("Conectando...")
    while not wlan.isconnected():
        sleep_ms(1000)

config = wlan.ifconfig()
print(f"Conectado con ip {config[0]}")

#--- Conexión a MQTT --------------------------------#
def callback(topic, msg):
    topic = topic.decode()
    msg = msg.decode()
    print(f"Llegó {msg} de {topic}")

    # De acuerdo al mensaje recibido, encender o apagar el pin 2
    if topic == mqtt_topic_1 and msg == "ON":
        pin_2.on()
    elif topic == mqtt_topic_1 and msg == "OFF":
        pin_2.off()

# Obtener la dirección MAC para el ID del cliente
mac = wlan.config('mac')
mac_address = ':'.join('{:02x}'.format(b) for b in mac)
print("Dirección MAC:", mac_address)

cliente = MQTTClient(mac_address, mqtt_server, user=mqtt_user, 
                        password=mqtt_password, keepalive=30)
print("Conectando a Adafruit IO...")
cliente.set_callback(callback)
cliente.connect(clean_session=False)
print("Conectado")
cliente.subscribe(mqtt_topic_1)

# Publicar en el servidor un valor aleatorio
def publisher(timer):
    temperature = urandom.randint(0, 50)
    print("Publicando", temperature, "°C")
    cliente.publish(mqtt_topic_2, str(temperature))

tim = Timer(0)
tim.init(period=10000, mode=Timer.PERIODIC, callback=publisher)

#--- Bucle infinito ----------------------------------#
while True:
    cliente.check_msg()
    sleep_ms(500)

## 11. Programación asincrónica: ejecutar múltiples tareas

[MicroPython: ESP32/ESP8266 Asynchronous Programming – Run Multiple Tasks](https://randomnerdtutorials.com/micropython-esp32-esp8266-asynchronous-programming/)

In [ ]:
# Rui Santos & Sara Santos - Random Nerd Tutorials
# Complete project details at https://RandomNerdTutorials.com/micropython-esp32-esp8266-asynchronous-programming/

import asyncio
from machine import Pin

green_led_pin = 14
green_led = Pin(green_led_pin, Pin.OUT)
blue_led_pin = 12
blue_led = Pin(blue_led_pin, Pin.OUT)

# Define coroutine function
async def blink_green_led():
    while True:
        green_led.value(not green_led.value() )
        await asyncio.sleep(0.5) 

# Define coroutine function
async def blink_blue_led():
    while True:
        blue_led.value(not blue_led.value())
        await asyncio.sleep(1)

# Define the main function to run the event loop
async def main():
    # Create tasks for blinking two LEDs concurrently
    asyncio.create_task(blink_green_led())
    asyncio.create_task(blink_blue_led())

# Create and run the event loop
loop = asyncio.get_event_loop()  
loop.create_task(main())  # Create a task to run the main function
loop.run_forever()  # Run the event loop indefinitely